In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(1)
seed(1)

Using TensorFlow backend.


ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 


In [2]:
dataset = pd.read_csv("C:\\Users\\anshu\\Desktop\\train.csv")
dataset.head()

,date,item,sales
0,01-01-2013,1,13
1,02-01-2013,1,11
2,03-01-2013,1,14
3,04-01-2013,1,13
4,05-01-2013,1,10


In [3]:
daily_sales = dataset.groupby('date', as_index=False)['sales'].sum()

In [4]:
daily_sales

,date,sales
0,01-01-2013,13696
1,01-01-2014,15556
2,01-01-2015,17226
3,01-01-2016,20158
4,01-01-2017,23709
5,01-02-2013,17473
6,01-02-2014,21074
7,01-02-2015,23395
8,01-02-2016,17009
9,01-02-2017,20612


In [5]:
item_daily_sales = dataset.groupby(['item', 'date'], as_index=False)['sales'].sum()
item_daily_sales

,item,date,sales
0,1,01-01-2013,133
1,1,01-01-2014,109
2,1,01-01-2015,143
3,1,01-01-2016,182
4,1,01-01-2017,205
5,1,01-02-2013,148
6,1,01-02-2014,171
7,1,01-02-2015,205
8,1,01-02-2016,130
9,1,01-02-2017,176


In [6]:
dataset = dataset[(dataset['date'] >= '2017-01-01')]
train = dataset[(dataset['date'] >= '2017-01-01')]
train_gp = train.sort_values('date').groupby(['item', 'date'], as_index=False)
train_gp = train_gp.agg({'sales':['mean']})
train_gp.columns = ['item', 'date', 'sales']
train_gp.head()

,item,date,sales
0,1,21-01-2013,9.4
1,1,21-01-2014,10.3
2,1,21-01-2015,14.7
3,1,21-01-2016,13.9
4,1,21-01-2017,17.3


In [7]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg
window = 29
lag = 90
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)
series.head()

,item(t-29),sales(t-29),item(t-28),sales(t-28),item(t-27),sales(t-27),item(t-26),sales(t-26),item(t-25),sales(t-25),...,item(t-3),sales(t-3),item(t-2),sales(t-2),item(t-1),sales(t-1),item(t),sales(t),item(t+90),sales(t+90)
29,1.0,9.4,1.0,10.3,1.0,14.7,1.0,13.9,1.0,17.3,...,1.0,30.1,1.0,30.7,1.0,29.9,1,28.7,1.0,19.7
30,1.0,10.3,1.0,14.7,1.0,13.9,1.0,17.3,1.0,13.6,...,1.0,30.7,1.0,29.9,1.0,28.7,1,27.5,1.0,12.6
31,1.0,14.7,1.0,13.9,1.0,17.3,1.0,13.6,1.0,19.1,...,1.0,29.9,1.0,28.7,1.0,27.5,1,23.6,1.0,14.0
32,1.0,13.9,1.0,17.3,1.0,13.6,1.0,19.1,1.0,20.3,...,1.0,28.7,1.0,27.5,1.0,23.6,1,28.5,1.0,18.2
33,1.0,17.3,1.0,13.6,1.0,19.1,1.0,20.3,1.0,18.5,...,1.0,27.5,1.0,23.6,1.0,28.5,1,31.8,1.0,18.8


In [8]:
last_item = 'item(t-%d)' % window
series = series[(series['item(t)'] == series[last_item])]

columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['item']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['item(t)'], axis=1, inplace=True)

labels_col = 'sales(t+%d)' % 90
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

In [9]:
X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (17856, 30)
Validation set shape (11904, 30)


,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
24257,36.7,38.8,49.1,60.0,69.1,45.1,61.7,64.5,55.0,53.6,...,42.9,51.9,36.2,45.5,53.5,29.4,39.2,37.6,45.8,56.1
13365,68.2,52.8,66.9,79.4,87.6,66.0,76.4,86.2,111.5,80.3,...,79.1,93.5,109.6,128.5,95.5,88.7,104.3,71.2,93.9,102.3
13468,102.7,102.2,43.6,56.4,65.9,82.4,51.8,52.2,63.0,41.2,...,83.7,99.4,77.2,86.4,103.5,87.9,83.2,92.1,68.7,85.2
17164,102.4,113.1,50.5,58.9,65.9,72.3,86.9,55.2,68.2,74.0,...,87.4,97.3,73.4,90.6,104.2,80.3,95.3,91.9,112.9,76.6
14327,24.8,34.4,37.7,30.4,32.2,18.4,17.0,20.8,22.2,25.8,...,22.3,30.0,30.3,38.0,43.5,28.3,30.2,35.5,31.4,34.5


In [11]:
subsequences = 2
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))
timesteps = X_train_series.shape[1]//subsequences
X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, timesteps, 1))
X_valid_series_sub = X_valid_series.reshape((X_valid_series.shape[0], subsequences, timesteps, 1))

In [12]:
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation='relu'))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mse', optimizer='adam')

In [13]:
model_cnn_lstm.fit(X_train_series_sub, Y_train, 3, verbose=0)

In [14]:
yhat = model_cnn_lstm.predict(X_valid_series_sub, verbose=0)

In [15]:
yhat

array([[77.18835 ],
       [23.808765],
       [88.29585 ],
       ...,
       [70.56859 ],
       [30.574202],
       [61.038532]], dtype=float32)